In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
#### INSTRUCTIONS FOR I/O (PLEASE READ) #######
# Input data files are available in the read-only "../input/" (relative) or '/kaggle/input'(absolute) directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
input_path = '2024-flame-ai-challenge/dataset/'
output_path = 'working/'

In [2]:
test_df = pd.read_csv(os.path.join(input_path,'test.csv'))
test_df.head()

,id,u,alpha,Nt,Nx,Ny,theta_filename,ustar_filename,xi_filename
0,219547,5,2.5,5,113,32,theta_K_id219547.dat,ustar_ms-1_id219547.dat,xi_id219547.dat
1,167403,5,2.5,5,113,32,theta_K_id167403.dat,ustar_ms-1_id167403.dat,xi_id167403.dat
2,225258,5,2.5,5,113,32,theta_K_id225258.dat,ustar_ms-1_id225258.dat,xi_id225258.dat
3,890407,5,25.0,5,113,32,theta_K_id890407.dat,ustar_ms-1_id890407.dat,xi_id890407.dat
4,352206,5,25.0,5,113,32,theta_K_id352206.dat,ustar_ms-1_id352206.dat,xi_id352206.dat


In [3]:
#gets test set input
def getTestX(idx):
    csv_file = test_df.reset_index().to_dict(orient='list')
    dir_path = os.path.join(input_path, "test")
    id = csv_file['id'][idx]
    nt, Nx, Ny = csv_file['Nt'][idx], csv_file['Nx'][idx], csv_file['Ny'][idx]
    theta = np.fromfile(os.path.join(dir_path, csv_file['theta_filename'][idx]), dtype="<f4").reshape(nt, Nx, Ny)
    xi_f = np.fromfile(os.path.join(dir_path, csv_file['xi_filename'][idx]), dtype="<f4").reshape(nt, Nx, Ny)
    uin  = np.array(csv_file['u'][idx])
    alpha = np.array(csv_file['alpha'][idx])
    uin = np.full_like(theta,uin)
    alpha = np.full_like(theta,alpha)

    X = np.stack([theta,xi_f,uin,alpha],axis=-1) # (t, Nx, Ny, c) 
    X = torch.tensor(X)
    return id,X

#predicts with input
def predict(idx,model):
    id,X = getTestX(idx)
    X = X.unsqueeze(0)
    y_pred = model(X)
    return id,y_pred

#generates submission with model predictions already in SI units
def generate_submission(model):
    y_preds = {}
    ids = []
    for idx in range(len(test_df)):
        id, y_pred = predict(idx, model) 
        #WARNING tmp should be in SI units
        y_preds[id]= np.array(y_pred).flatten(order='C').astype(np.float32)
        ids.append(id)
    df = pd.DataFrame.from_dict(y_preds,orient='index')
    df['id'] = ids

    # move id to first column
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    #reset index
    df = df.reset_index(drop=True)

    return df


In [4]:
# create a torch model based on linear interpolation of fire spread
# REPLACE THIS WITH YOUR MODEL LOADER TO MAKE YOUR PREDICTIONS
class FireSpreadModel(nn.Module):
    def __init__(self, n_predictions=5):
        super(FireSpreadModel, self).__init__()
        self.n_predictions = n_predictions
        # Constants
        self.R_2_0 = 0.89492756  # m/s
        self.R_6_0 = 1.7765957
        self.R_2_10 = 2.009707
        
        # Calculate slopes for u10 and slope effects
        self.k_u = (self.R_6_0 - self.R_2_0) / (6 - 2)
        self.k_slope = (self.R_2_10 - self.R_2_0) / (10 - 0)
    
    def find_initial_location(self, indicator):
        initial_index = -1
        for i in range(indicator.shape[1]): 
            if indicator[-1,i,0] > 0:
                initial_index = i
                break
        return initial_index
    
    def forward(self, data):
        """
        This model takes in:
        data:(time_steps, nx, ny, 4) tensor containing the input data
        Outputs:
        - xi_f_predictions: (n_predictions, nx, ny)
        """
        indicator = data[:, :, :, :, 1].squeeze()
        u10 = data[:, :, :, :, 2].squeeze()
        u10 = u10.mean()                         # we are only interested in the u as single value
        slope = data[:, :, :, :, 3].squeeze()    # we are only interested in the slope as single value
        slope = slope.mean()
        # Calculate displacement per second
        displ_per_second = self.R_2_0 + self.k_u * (u10 - 2) + self.k_slope * (slope - 0)
        
        initial_index = self.find_initial_location(indicator)

        xi_f = initial_index*8 + displ_per_second
        indicator_predictions = torch.zeros((self.n_predictions, indicator.shape[1], indicator.shape[2]))

        # Predict the next locations
        for i in range(1, self.n_predictions):
            indicator_predictions[i,int(xi_f/8),:] = 1
            xi_f = xi_f + displ_per_second
            

        return indicator_predictions # (20, nx, ny)

In [5]:
model = FireSpreadModel(n_predictions=20)
# predict(0, model)
df = generate_submission(model)
df.to_csv(os.path.join(output_path, 'submission.csv'),index=False)
print('Generating Submission file ... completed' )

Generating Submission file ... completed
